In [1]:
import glob
import numpy as np 
import time
import math
import random
from scipy import linalg as LA
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.svm import NuSVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
import seaborn as sns
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import statsmodels.api as sm
import matplotlib.pyplot as plt
from termcolor import colored
from matplotlib import colors
import pickle
from datetime import datetime
import os
from sklearn.svm import LinearSVC
from hmmlearn import hmm
from pprint import pprint 
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

## Do not run the following blocks until it is mentioned at very bottom

In [2]:
N = 182 # number of users
J = glob.glob('Geolife Trajectories 1.3/**/', recursive=True)[2:]
K = [J[2*i] for i in range(N)]
F = [K[i][30:33] for i in range(N)]
int1 = np.vectorize(int)
folder_numbers = int1(F)
I = glob.glob('Geolife Trajectories 1.3/**/*.plt', recursive=True)
#np.sort(folder_numbers), len(folder_numbers)

In [3]:
def read_file(file_name):
    data = []
    c = 0
    with open(file_name, "r") as f:
        for line in f:
            c = c + 1
            if c > 6:
                item = line.strip().split(",")
                if len(item) == 7:
                    data.append([float(item[0]), float(item[1]), float(item[4]), None])
    return np.array(data)

In [4]:
# Test
path = glob.glob('Geolife Trajectories 1.3/Data/010/Trajectory/*.plt', recursive=True)
print(len(path)) 
print(read_file(path[5]))

161
[[44.589055 129.603843 39299.2951736111 None]
 [44.589057 129.603843 39299.2951851852 None]
 [44.588792 129.60388 39299.2958217593 None]
 ...
 [44.196218 125.503752 39299.7169675926 None]
 [44.193637 125.501802 39299.717037037 None]
 [44.180203 125.49309 39299.7174768519 None]]


# Reading Labels

In [5]:
I_text = glob.glob('Geolife Trajectories 1.3/**/*.txt', recursive=True)
len(I_text)

69

In [6]:
labeled_users = [I_text[i][30:33] for i in range(len(I_text))]
print(labeled_users) 
print(len(labeled_users))

['104', '161', '102', '105', '167', '084', '085', '082', '076', '078', '065', '091', '096', '062', '053', '098', '052', '097', '064', '174', '129', '116', '111', '118', '144', '175', '126', '110', '128', '117', '153', '154', '136', '100', '107', '138', '163', '106', '139', '101', '108', '089', '087', '073', '080', '020', '081', '075', '086', '088', '010', '021', '092', '059', '068', '056', '069', '067', '058', '060', '112', '115', '124', '170', '141', '179', '125', '114', '147']
69


In [7]:
# create 'daysDate' function to convert start and end time to a number of days

def days_date(time_str):
    date_format = "%Y/%m/%d %H:%M:%S"
    current = datetime.strptime(time_str, date_format)
    date_format = "%Y/%m/%d"
    bench = datetime.strptime('1899/12/30', date_format)
    no_days = current - bench
    delta_time_days = no_days.days + current.hour / 24.0 + current.minute / (24. * 60.) + current.second / (24. * 3600.)
    return delta_time_days

In [8]:
# Change Mode Name to Mode index
Mode_Index = {"walk": 0, "bike": 1, "bus": 2, "car": 3, "taxi": 3, "subway": 4, 
              "railway": 4, "train": 4, "motorcycle": 5, "run": 5, "boat": 5, 
              "airplane": 5, "other": 5}

# Modes are the modes that we use here
Modes = ['walk', 'bike', 'bus', 'driving', 'train', 'others']

# We will remove 'others'

In [9]:
def get_labels(path):
    
    labels_data = []
    c = 0
    with open(path, "r") as f:
        for line in f:
            if c > 0:
                item = line.strip().split("\t")
                if len(item) == 3:
                    labels_data.append([float(days_date(item[0])), 
                                        float(days_date(item[1])), Mode_Index[item[2]]])
            else:
                c += 1
    labels_data = np.array(labels_data)

    return set(labels_data[:,-1]), labels_data

In [10]:
# Test
p = glob.glob('Geolife Trajectories 1.3/Data/010/labels.txt')
get_labels(p[0])

({0.0, 2.0, 3.0, 4.0, 5.0},
 array([[3.92594809e+04, 3.92594864e+04, 2.00000000e+00],
        [3.95356201e+04, 3.95356667e+04, 4.00000000e+00],
        [3.95356667e+04, 3.95359181e+04, 4.00000000e+00],
        ...,
        [3.97894883e+04, 3.97895163e+04, 2.00000000e+00],
        [3.97895164e+04, 3.97895174e+04, 0.00000000e+00],
        [3.97895178e+04, 3.97895260e+04, 3.00000000e+00]]))

### Correcting date and time of label dataset according to those of trajectories

In [11]:
n = len(I_text)
users_labels = [0] * n
E = []
for i in range(n):
    users_labels[i] = get_labels(I_text[i])[1]
    E.append(I_text[i][30:33])
users_labels = np.array(users_labels)
#users_labels[0]

### Determining all transportation modes in data

In [12]:
S = set()
for path in I_text:
    S = S.union(get_labels(path)[0])
print(S)

{0.0, 1.0, 2.0, 3.0, 4.0, 5.0}


In [13]:
Labels = [0,1,2,3,4,5, None]
len(Labels)

7

# Adding transportation mode to trajectories

In [14]:
# Test
V = glob.glob('Geolife Trajectories 1.3/Data/010/Trajectory/*.plt', recursive = True)
pp = 'Geolife Trajectories 1.3/Data/010/labels.txt'
#np.where(get_labels(pp)[1][:,2] != None)[0]

In [15]:
print("one line from labels data:", get_labels(pp)[1][0])
print("one line from a trajectory:", read_file(V[0])[0])

one line from labels data: [3.92594809e+04 3.92594864e+04 2.00000000e+00]
one line from a trajectory: [39.138159 117.217108 39805.9617476852 None]


In [16]:
def add_labels(trajectory, labels):
    count = 0
    epn = 2/(24 * 3600)
    for i in range(len(labels)):
        a1 = np.where((trajectory[:,2] >= labels[i][0] - epn))[0]
        b1 = np.where((trajectory[:,2] <= labels[i][0] + epn))[0]
        c1 = list(set(a1).intersection(b1))
        
        a2 = np.where((trajectory[:,2] >= labels[i][1] - epn))[0]
        b2 = np.where((trajectory[:,2] <= labels[i][1] + epn))[0]
        c2 = list(set(a2).intersection(b2))
        
        if len(c1) > 0 and len(c2) > 0:
            trajectory[:,3][c1[0]:c2[-1]+1] = labels[i][-1]
            count += 1
    return trajectory, count

In [17]:
# Test
V = glob.glob('053/Trajectory/*.plt', recursive = True)
pp = '053/labels.txt'
add_labels(read_file(V[0]), get_labels(pp)[1])[0][:,-1]

array([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0,
       2.0, None, None, None, None, None, None], dtype=object)

In [18]:
def add_label_to_user(user_path, labels):
    c = 0
    user = []
    for i in range(len(user_path)):
        A = add_labels(read_file(user_path[i]), labels)
        user.append(A[0])
        c += A[1]
        
    return np.array(user), c

In [23]:
start_time = time.time()
c1 = 0 
c2 = 0
data = [0] * n
for i in range(n):
    print(i)
    user_path = glob.glob('Geolife Trajectories 1.3/Data/'+E[i]+'/trajectory/*.plt', 
                          recursive=True)
    a = add_label_to_user(user_path, users_labels[i])
    data[i] = np.array([a[1], len(users_labels[i]), a[0]])
    print('a[1], users_labels[i] =', a[1], len(users_labels[i]))
    c1 += a[1]
    c2 += len(users_labels[i])
    
data = np.array(data)
print("c1, c2 =", c1, c2)
print(time.time() - start_time)

0
a[1], users_labels[i] = 0 88
1
a[1], users_labels[i] = 20 38
2
a[1], users_labels[i] = 50 57
3
a[1], users_labels[i] = 25 32
4
a[1], users_labels[i] = 648 944
5
a[1], users_labels[i] = 277 422
6
a[1], users_labels[i] = 788 1298
7
a[1], users_labels[i] = 98 119
8
a[1], users_labels[i] = 3 11
9
a[1], users_labels[i] = 58 94
10
a[1], users_labels[i] = 63 206
11
a[1], users_labels[i] = 56 170
12
a[1], users_labels[i] = 82 119
13
a[1], users_labels[i] = 431 782
14
a[1], users_labels[i] = 16 20
15
a[1], users_labels[i] = 23 28
16
a[1], users_labels[i] = 220 425
17
a[1], users_labels[i] = 17 28
18
a[1], users_labels[i] = 42 68
19
a[1], users_labels[i] = 0 7
20
a[1], users_labels[i] = 15 16
21
a[1], users_labels[i] = 0 3
22
a[1], users_labels[i] = 13 36
23
a[1], users_labels[i] = 0 5
24
a[1], users_labels[i] = 4 9
25
a[1], users_labels[i] = 11 12
26
a[1], users_labels[i] = 353 468
27
a[1], users_labels[i] = 0 20
28
a[1], users_labels[i] = 700 937
29
a[1], users_labels[i] = 9 15
30
a[1], user

In [29]:
np.where(data[:,0] == 0)[0]

array([ 0, 19, 21, 23, 27, 31, 33, 37, 49, 53, 62, 68])

In [34]:
idx = np.where(data[:,0] != 0)[0]
data = data[idx]
len(data)

57

# Removing unlabeled data

In [92]:
Data = [0] * len(data)
for i in range(len(data)):
    Data[i] = []
    for j in range(len(data[i][2])):
        I = np.where((data[i][2][j][:,3] != None) & (data[i][2][j][:,3] != 5) )[0]
        if len(I) > 0:
            Data[i].append(data[i][2][j][I])
    Data[i] = np.array(Data[i])
Data = np.array(Data)

In [128]:
print("Total number of trajectories in the cleaned data:", 
      sum([len(Data[i]) for i in range(len(Data))]))

Total number of trajectories in the cleaned data: 3572


# Checking labels of cleaned data

In [129]:
Labels_final1 = []
Labels_final = set([])
for i in range(len(Data)):
    Labels_final1.append([set(Data[i][j][:,3]) for j in range(len(Data[i]))])
    for j in range(len(Data[i])):
        Labels_final = Labels_final.union(Labels_final1[i][j])
Labels_final

{0.0, 1.0, 2.0, 3.0, 4.0}

# Creating csv files

In [208]:
def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

In [199]:
DF1 = pd.DataFrame(Data[0][0]) 
DF1.to_csv(r'labeled csv Geolife/data['+str(1)+'].csv', index=False)

In [218]:
for i in range(len(Data)):
    createFolder('labeled csv Geolife/'+str(i))
    for j in range(len(Data[i])):
        DF = pd.DataFrame(Data[i][j]) 
        DF.to_csv(r'labeled csv Geolife/'+str(i)+'/'+str(i)+'_'+str(j)+'.csv', index=False)